# Blog

> My blog!

In [36]:
#| default_exp blog

In [48]:
#| export
from fasthtml.common import *
from monsterui.all import *
from utils import *
from pathlib import Path
import glob, yaml
from fastcore.ansi import ansi2html
from html import escape
from execnb.nbio import read_nb
from execnb.shell import render_outputs
from nbdev.process import extract_directives

In [49]:
#| export
ar = APIRouter(prefix="/blog", body_wrap=layout)

In [50]:
#| export
path = Path()
if IN_NOTEBOOK: path = Path('../')

In [51]:
#| export
def get_notebooks():
    notebooks = []
    for nb in (path/'posts').rglob('*.ipynb'):
        if not nb.name.startswith('_') and '.ipynb_checkpoints' not in str(nb):
            notebooks.append(str(nb))
    return notebooks

In [52]:
#| export
def get_meta(nb): return yaml.safe_load(nb.cells[0].source.split('---')[1])

In [53]:
#| export
def get_nb_lang(nb): 
    try: return nb['metadata']['kernelspec']['language']
    except Exception as e: return 'python'

In [54]:
from IPython.utils.text import strip_ansi
from html import escape
def _pre(s, xtra=''): return f"<pre {xtra}><code>{escape(s)}</code></pre>"
def _strip(s): return strip_ansi(escape(s))

In [55]:
#| export
def render_code_output(cell, directives, lang='python', pygments=False, wrapper=Footer):
    if not cell.outputs: return ''
    if 'include:' in directives and directives['include:'][0] =='false': return ''

    def render_output(out):
        otype = out['output_type']
        if otype == 'stream':
            txt = ansi2html(''.join(out["text"]))
            xtra = '' if out['name']=='stdout' else "class='stderr'"
            is_err = '<span class' in txt
            return Safe(f"<pre {xtra}><code class='{'nohighlight hljs' if is_err else ''}'>{txt}</code></pre>")
        elif otype in ('display_data','execute_result'):
            data = out['data']
            _g = lambda t: ''.join(data[t]) if t in data else None
            if d := _g('text/html'): return Safe(apply_classes(d))
            if d := _g('application/javascript'): return Safe(f'<script>{d}</script>')
            if d := _g('text/markdown'): return render_md(d)
            if d := _g('text/latex'): return Safe(f'<div class="math">${d}$</div>')
            if d := _g('image/jpeg'): return Safe(f'<img src="data:image/jpeg;base64,{d}"/>')
            if d := _g('image/png'): return Safe(f'<img src="data:image/png;base64,{d}"/>')
            if d := _g('text/plain'): return escape(d)
            if d := _g('image/svg+xml'): return Safe(d)
        return ''
    
    res = Div(*map(render_output, cell.outputs))
    if res: return wrapper(res)

In [81]:
#| export
def render_code_input(cell, directives, lang='python'):
    code = f'''```{lang}\n{cell.source}\n```\n'''
    if 'include:'   in directives and directives['include:'][0]   =='false': return ''
    if 'echo:'      in directives and directives['echo:'][0]      =='false': return ''
    if 'code-fold:' in directives and directives['code-fold:'][0] == 'true':
        return Details(Summary("See Code"),render_md(code))
    return render_md(code)

In [ ]:
#| export
def remove_directives(cell):
    "Remove #| directives from start of cell"
    lines = cell.source.split('\n')
    while lines and lines[0].startswith('#|'): lines.pop(0)
    cell.source = '\n'.join(lines)

In [85]:
#| export
def render_nb(nb):
    "Render a notebook as a list of html elements"
    res = []
    lang = get_nb_lang(nb)
    meta = get_meta(nb)
    res.append(Div(H1(meta['title']), Subtitle(meta['description']), cls='my-9'))
    for cell in nb.cells[1:]:  
        if cell['cell_type']=='code':
            directives = extract_directives(cell)
            remove_directives(cell)
            _output = render_code_output(cell, directives)
            res.append(render_code_input(cell, directives))
            res.append(Card(_output) if _output else '')
        elif cell['cell_type']=='markdown':
            res.append(render_md(cell.source))
    return res

In [69]:
nb = read_nb('../app/posts/MeanShiftFromScratch.ipynb')

In [70]:
_cell = nb.cells[3]
_cell

```json
{ 'cell_type': 'code',
  'execution_count': 3,
  'id': '35e9e944-64a9-4afc-a267-448aa9350b02',
  'idx_': 3,
  'metadata': {},
  'outputs': [],
  'source': '#| code-fold: true\n'
            'import math, random, matplotlib.pyplot as plt, operator, torch\n'
            'from functools import partial\n'
            'from fastcore.all import *\n'
            'from torch.distributions.multivariate_normal import '
            'MultivariateNormal\n'
            'from torch import tensor\n'
            'import numpy as np'}
```

In [ ]:
#| export
@ar
def index(): 
    metas = []
    for fpath in get_notebooks():
        _meta = get_meta(read_nb(fpath))
        _meta['fpath'] = fpath
        metas.append(_meta)
    metas.sort(key=lambda x: x['date'], reverse=True)
    return Div(
        Div(H1("My Blog", cls="mb-2"),
            Subtitle("Thoughts, tutorials and technical writings", cls=TextT.gray+TextT.lg),
            cls="text-center py-8"),
        Div(Grid(*map(blog_card, metas), cols=1),
            cls="max-w-4xl mx-auto px-4"))

In [ ]:
#| export
@ar
def blog_post(fpath:str): return render_nb(read_nb(fpath))

In [ ]:
#| export
def blog_card(meta):
    def Tags(cats): return DivLAligned(map(Label, cats))
    
    return Card(
        DivLAligned(
            A(Img(src=meta.get('image',''), style="width:200px"), href=blog_post.to(fpath=meta['fpath'])),
            Div(cls='space-y-3 w-full')(
                H4(meta['title']),
                P(meta['description']),
                DivFullySpaced(map(Small, [meta['author'], meta['date']]), cls=TextT.meta),
                DivFullySpaced(
                    Tags(meta['categories']),
                    A("Read", cls=('uk-btn', ButtonT.primary,'h-6'), href=blog_post.to(fpath=meta['fpath']))))),
        cls=CardT.hover)